# 优化器的模式

        优化器的模式用户决定Oracle解析目标SQL时所用优化器的类型，以及决定使用CBO时计算成本的侧重点。优化器的模式不同，CBO计算成本值的方法也不同。
**优化器模式参数（OPTIMIZER_MODE）**


1、RULE

        用RBO来解析目标SQL。
2、CHOOSE

        Oracle在解析目标SQL时，如果有一个表对象有统计信息就用CBO，否则用RBO。
3、FIRST_ROWS_n(n= 10, 100, 1000)
    
        Oracle会使用CBO解析目标SQL，在计算成本值时的侧重点在于以最快的响应速度返回头n条记录。
4、FIRST_ROWS

        Oracle在解析目标SQL时会联合使用CBO和RBO。解析时采用CBO，侧重点在于以最快的响应速度返回记录；当出现一些特定的情况时，会用RBO中的一些内置规则来选取执行计划而不考虑成本。不如通过使用索引来避免排序。
5、ALL_ROWS

        Oracle用CBO解析目标SQL，此时计算成本值时的侧重点在于最佳的吞吐量（即最小的系统I/O和CPU资源的消耗量）

# 结果集（Row Source）

        结果集是指目标SQL执行计划的执行步骤对应的输出结果集。

# 访问数据的方法

## 访问表的方法

### 全表扫描

        全表扫描是指Oracle在访问目标表里的数据时，会从该表所占用的第一个区（EXTENT）的第一个块（BLOCK）开始扫描，一直扫描到该表的高水位线（HWM,High Water Mark），这段范围内所有的数据块Oracle都必须读到。Oracle会对这期间读到的所有数据施加目标SQL的where条件过滤，最后返回满足要求的数据。
        Oracle在做全表扫描操作时会使用多块读，在目标表数据量不大时执行效率是非常高的。全表扫描最大的问题是执行时间不稳定、不可控，会随着目标表数据量的递增而递增。
        高水位线特性的副作用：即使DELETE删光了目标表里的所有数据，高水位线还是会在原来的位置，全表扫描的时候还是会扫描高水位线下所有的数据块。

### ROWID扫描

        ROWID表示的是Oracle中数据行记录所在的物理存储位置，同数据块中的记录一一对应。Oracle通过ROWID去定位并访问数据的方法叫ROWID扫描。
**ROWID的两层含义**

    1、根据SQL语句中输入的ROWID值直接去访问对应的行记录
    2、通过访问相关索引，再根据索引得到的ROWID访问数据

> 拓展DBMS_ROWID包

        1、根据给定参数返回一个rowid
```
FUNCTION rowid_create(rowid_type    IN NUMBER,--rowid类型，0：restricted，1：extended
                      object_number IN NUMBER,--对象id
                      relative_fno  IN NUMBER,--关联文件
                      block_number  IN NUMBER,--数据块id
                      row_number    IN NUMBER)--行号 
                      RETURN ROWID;
```
        2、根据rowid返回相关参数
```
PROCEDURE rowid_info(rowid_in      IN  ROWID,--rowid
                     rowid_type    OUT NUMBER,--rowid类型
                     object_number OUT NUMBER,--对象id
                     relative_fno  OUT NUMBER,--相关文件
                     block_number  OUT NUMBER,--数据块id
                     row_number    OUT NUMBER,--行号
                     ts_type_in    IN VARCHAR2 DEFAULT 'SMALLFILE');--表空间类型，BIGFILE：大文件表空间，SMALLFILE：传统表空间
```
        3、通过rowid返回rowid类型
```
function rowid_type(row_id IN rowid)--rowid
                        return number;
```
        4、通过rowid提取数据文件编号
```
function rowid_object(row_id IN rowid)--rowid
                        return number;
```
        5、通过rowid提取相关文件编号
```
FUNCTION rowid_relative_fno(row_id     IN ROWID,--rowid
                            ts_type_in IN VARCHAR2 DEFAULT 'SMALLFILE')--表空间类型
  RETURN NUMBER;
```
        6、通过rowid提取数据块编号
```
FUNCTION rowid_block_number(row_id     IN ROWID,--rowid
                            ts_type_in IN VARCHAR2 DEFAULT 'SMALLFILE')--表空间类型
  RETURN NUMBER;
```
        7、通过rowid提取行号
```
function rowid_row_number(row_id IN rowid)--rowid
                        return number;
```
        8、通过rowid和给定表，提取相关文件编号
```
FUNCTION rowid_to_absolute_fno(row_id      IN ROWID,--rowid
                               schema_name IN VARCHAR2,--拥有者
                               object_name IN VARCHAR2) --对象名
                               RETURN NUMBER;
```
        9、转换一个restricted rowid为一个extended rowid
```
※ 如果原始的rowid存储在列中，转换的 就是internal类型；
※ 如果原始的rowid是以字符串形式存储的，那转换的就是external类型
FUNCTION rowid_to_extended(old_rowid       IN ROWID,--rowid
                           schema_name     IN VARCHAR2,--拥有者
                           object_name     IN VARCHAR2,--对象名
                           conversion_type IN INTEGER)--转换类型，0：转换restricted internal rowid为extended格式
                                                      --转换类型，1：转换restricted external rowid为extended格式
```
        10、转换一个exteneded的rowid为一个restricted的rowid
```
※ restricted的rowid格式为BBBBBBB.RRRR.FFFFF, 
※ BBBBBBB 代表block
※ RRRR 代表在block中的行号，从0开始
※ FFFFF 代表文件号。
※ 这个包可以使用rowid或者rowid转换类型（ROWID_CONVERT_INTERNAL (0)和ROWID_CONVERT_EXTERNAL (1)）
FUNCTION rowid_to_restricted(old_rowid       IN ROWID,--rowid
                             conversion_type IN INTEGER)--rowid_convert_internal  := 0;
                                                        --rowid_convert_external  := 1;
                             RETURN ROWID;
```
        11、验证rowid 有效性
```
FUNCTION rowid_verify(rowid_in        IN ROWID,--rowid
                      schema_name     IN VARCHAR2,--拥有者
                      object_name     IN VARCHAR2,--对象名
                      conversion_type IN INTEGER)--rowid_convert_internal := 0;
                                                 --rowid_convert_external := 1;
                      RETURN NUMBER;
```